# Natural Language Processing Intro

> Some of the examples in this notebook are adapted from the following sources:
> * NVIDIA DLI: [Building Transformer-Based Natural Language Processing Applications](https://learn.nvidia.com/courses/course-detail?course_id=course-v1:DLI+C-FX-03+V3). This is an instructor-led course that I teach periodically--let me know if you have a group that would be interested in taking this workshop.
> * LinkedIn Learning: [Introduction to Transformer Models for NLP by Sinan Ozdemir](https://www.linkedin.com/learning/introduction-to-transformer-models-for-nlp/introduction). This is a great course, and like all LinkedIn Learning courses, is free to UF students, faculty and staff.


Natural Language Processing (NLP) is a large field of AI with many related but somewhat distinct sub-disciplines. We won't have time to look at all of these, but you are most likely somewhat familiar with many of the applications.

Some NLP Tasks:
* Summary generation, information extractions
* Translation (language to language)
* Transcription (speech to sample_text)
* Auto-completion
* Sentiment Analysis
* Intent Detection
* Voice assistant
* Document retrieval
* And the BIG one these days, **Chat**, automated writing, dialog generation, question answering, etc., the kinds of things that ChatGPT is known for. 

## Ambiguity in language

NLP is not a simple task, and until deep learning, was quite limited in its abilities. As with most fields in AI, there is a long history [dating back to the 1950s](https://en.wikipedia.org/wiki/Natural_language_processing). Part of the challenge is that human language tends to be ambiguous, and recognizing words is really only the start of inferring meaning. Take for example, this sentence:

   > The boy saw a man with a telescope
   
   * Who had the telescope?
   
More consample_text is needed to answer this. Yet, consample_text is not always there, and even when it is, it can be a challenge for NLP methods (and even human readers at times).

## Tokenization

One of the challenges of NLP is representing language as numbers--remember, computers and the ML/AI systems that we have primarily deal with numbers. This was relatively easy for computer vision problems because we took the pixel intensities of an image and fed those into our models. But what should we do with speech, words, and sample_text?

The process of converting sample_text to numerical representation is called **tokenization**. There are many methods of tokenization, but the idea is to break sample_text into itemizable components--tokens.

Tokens can be words, letters, word fragments, or even sentences.


## The BERT WordPiece Tokenizers

From the NVIDIA Course:
> Tokenization splits a word, phrase, or larger sample_text section into individual characters, words, or subwords.  For example, the word "tokenization" could be split in a number of ways:
>
> * Characters: 't', 'o', 'k', 'e', 'n', 'i', 'z', 'a', 't', 'i', 'o', 'n'
> * Words: 'tokenization'
> * Subwords: 'token', '##ization'
>
> The idea is to create a vocabulary of tokens from a sample_text corpus, which can then be trained in a language model to characterize language relationships between the tokens.  Whether this is done by character, word, or subword affects the complexity of the problem.
>
> Tokenization by characters has the advantage of a very limited number of tokens to deal with, but these few tokens are not very meaningful by themselves and long sequences of tokens are required to represent sample_text.  Tokenization by words results in a very large vocabulary size and requires separate tokens for very similar words, which in turn requires more training to determine their relationships to each other.
>
> Tokenization by subwords is a solution that tries to balance these two. For example, the word "token" is a subword for "tokenization", "tokens", and "tokenize".  By splitting the words, the model learns similar meanings from the same root word more easily.  The size of the overall vocabulary required for understanding is less than required for word tokenization.

## WordPiece Algorithm
> The WordPiece algorithm was introduced in [this paper by Schuster and Nakajima](https://static.googleusercontent.com/media/research.google.com/ja//pubs/archive/37842.pdf).  To begin, the training data (corpus) is chosen, as well as the subword vocabulary size desired.   The algorithm iteratively determines optimal subwords for the body of sample_text and creates the vocabulary with assigned values.  The iterative steps are:
>
> 1. Split words into sequences of character tokens.
> 2. Build the language model on the training data using tokens from previous step.
> 3. Generate new unit tokens by combining two tokens with high likelihood in the language model and add the new token(s) to the vocabulary.
> 4. Repeat from step 2 until the token limit for the desired vocabulary is reached or the likelihood falls below some desired threshold.

Let's look at this in code...

In [ ]:
# Import nemo nlp collection 
from nemo.collections import nlp as nemo_nlp

# Import BERT
from nemo.collections.nlp.models import BERTLMModel

In [ ]:
# Check the list of pre-trained BERT language models
BERTLMModel.list_available_models()

> There are two pretrained BERT language models available with NeMo: 
> - `bertbaseuncased` model has 110 millions parameters in total with 12 Transformer blocks.
> - `bertlargeuncased` model has 340 millions parameters in total with 24 Transformer blocks.
> 
> For the sake of time and simplicity, we'll download the smaller variant, BERT Base. This could take a minute or two.

In [ ]:
# Download the pretrained BERT-based model
pretrained_model_name="bertbaseuncased"
model = BERTLMModel.from_pretrained(pretrained_model_name, strict=False) # Pass strict=False to avoid needing to specify
                                                                        # training parameters.

In [ ]:
# Take a look at the number of weights in the model
print(f'Number of weights in {pretrained_model_name}: {model.num_weights}')

In [ ]:
# Check available tokenizers
nemo_nlp.modules.get_tokenizer_list()

As I indicated, there are **a lot** of options on how to do tokenization...though in general, you will need to use what was used in training the model if you use a pre-trained model.

In [ ]:
# Get the bert-base-uncased tokenizer 
tokenizer_uncased = nemo_nlp.modules.get_tokenizer(tokenizer_name="bert-base-uncased")

In [ ]:
# Check the vocabulary size
print(f'The vocabulary size of {pretrained_model_name}: {tokenizer_uncased.vocab_size}')

In [ ]:
sample_sample_text = "Hello, my name is Matt. I live in Gainesville, FL."

In [ ]:
output_uncased=tokenizer_uncased.text_to_tokens(sample_sample_text)
print(f'Input sentence: {sample_sample_text}')
print(f'Tokenized sentence: {output_uncased}')

In [ ]:
# Get the bert-base-cased tokenizer 
tokenizer_cased = nemo_nlp.modules.get_tokenizer(tokenizer_name="bert-base-cased")

In [ ]:
# Encode the sample_text 
output_cased=tokenizer_cased.text_to_tokens(sample_sample_text)
print(f'Input sentence: {sample_sample_text}')
print(f'Tokenized sentence: {output_cased}')

> The BERT model [or any NLP model really] does not accept sample_text inputs, but rather their numerical index representations.
>
> We can check the vocabulary index of a word using the `tokenizer.sample_text_to_ids()` function. 
>
> Try it with the `bert-base-cased` tokenizer.

In [ ]:
# Index of the tokens Hello and hello using bert-base-cased tokenizer
print(f'Index of Hello: {tokenizer_cased.text_to_ids("Hello")}')
print(f'Index of hello: {tokenizer_cased.text_to_ids("hello")}')

In [ ]:
# Example of bert-base-cased tokenizer in a sentence
print(f'Input sentence: {sample_sample_text}')
print(f'Tokenized sentence: {output_uncased}')
print(f'Tokenized sentence: {tokenizer_uncased.text_to_ids(sample_sample_text)}')



## Embedding

The tokenization process takes care of converting words to numbers. This is a first step, but we want to consample_textualize the words--i.e., what does each word mean? Embedding goes a step further, representing words in n-dimensional vector space. This vector space has a lower dimensionality than the number of words in the vocabulary (as opposed to one-hot encoding) and results in words with similar meaning being closer in space than other words.

Here's an example figure showing some embeddings (taken from Renu Khandelwal's article [*Word Embeddings for NLP*](https://towardsdatascience.com/word-embeddings-for-nlp-5b72991e01d4):


![Image of word embeddings from https://towardsdatascience.com/word-embeddings-for-nlp-5b72991e01d4 ](images/word_embeddings_Renu_Khandelwal.png)

The embedding is learned in the training process. 

In [ ]:
import torch

# Set up the sentence we want to look at
sample_text = "Last night, my wireless mouse was eaten by an animal such as a mouse or rat. I need to order a new optical computer mouse."
input_sentence=torch.tensor([tokenizer_uncased.tokenizer(sample_text).input_ids]).cuda()
attention_mask=torch.tensor([tokenizer_uncased.tokenizer(sample_text).attention_mask]).cuda()

In [ ]:
# Show the tokenization for the sentence
print(f'Input sentence: {sample_text}')
output_uncased=tokenizer_uncased.ids_to_tokens(tokenizer_uncased.tokenizer(sample_text).input_ids)
print(f'Tokenized sentence: {output_uncased}')

# "mouse" tokens positions in the sample_text input
mouse_computer_1=6
mouse_animal=15
mouse_computer_2=27

print("Check that numbers line up:")
print(f'  Should be "mouse, mouse, mouse": {output_uncased[mouse_computer_1]}, {output_uncased[mouse_animal]}, {output_uncased[mouse_computer_2]}')


In [ ]:
# Get the embeddings for the pretrained model
hidden_states = model.bert_model(input_ids=input_sentence, token_type_ids=None, attention_mask=attention_mask)

In [ ]:
# This is the dimensionality of the embedding
len(hidden_states[0][0])

In [ ]:
from numpy import dot
from numpy.linalg import norm

def similarity_cosine(x,y):
    return dot(x,y)/(norm(x)*norm(y))

> We can visualize sample_text token embeddings obtained from the BERT models if we first reduce the dimensionality to 2D. 
[t-SNE](https://www.jmlr.org/papers/volume9/vandermaaten08a/vandermaaten08a.pdf) is a dimensionality reduction technique widely used for vector visualization in 2D or 3D, as it preserves the neighborhood distances.
>
> The following codeblocks uses the [sklearn](https://scikit-learn.org/stable/modules/generated/sklearn.manifold.TSNE.html) implementation of the t-SNE algorithm to reduce the dimensionality of BERT sample_text token embeddings from 768 to 2, and plots the 2D vectors. Note that as t-SNE is a stochastic process, the low dimensional embeddings will vary from one run to another. However, the neighborhood distances of tokens should remain more or less the same. 

In [ ]:
from sklearn.manifold import TSNE
import numpy as np

X=hidden_states.cpu().detach().numpy()
X_embedded = TSNE(n_components=2,metric='euclidean',  init='random', perplexity=7).fit_transform(X[0])
Tokens=tokenizer_uncased.ids_to_tokens(tokenizer_uncased.tokenizer(sample_text).input_ids)

# Annotate the different mouse tokens
Tokens[mouse_computer_1]="mouse_computer_1"
Tokens[mouse_animal]="mouse_animal"
Tokens[mouse_computer_2]="mouse_computer_2"

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
plt.figure(figsize=(15,10))
plt.plot(X_embedded[:,0],X_embedded[:,1], '.', color='black')
plt.plot([X_embedded[mouse_computer_1,0],X_embedded[mouse_animal,0]],[X_embedded[mouse_computer_1,1],X_embedded[mouse_animal,1]],color='red')
plt.plot([X_embedded[mouse_computer_1,0],X_embedded[mouse_computer_2,0]],[X_embedded[mouse_computer_1,1],X_embedded[mouse_computer_2,1]],color='green')

for i, txt in enumerate(Tokens):
    plt.annotate(txt, (X_embedded[i,0], X_embedded[i,1]), color='blue')

## Tokenizers and Embeddings are learned

> The tokenizer algorithm generates the vocabulary following variants of Top-K frequent words from the text corpus.
>
> The vocabulary size is limited because the training cost increases with the size of the vocabulary. Including all unique words from the text corpus into the vocabulary would explode the complexity of training beyond the capabilities of the tokenizer. For instance, the BERT model that was released in 2018, with a subword tokenizer algorithm called WordPiece, has a vocabulary limit of 30,000.
>
> How, then, do tokenizers deal with terms that are not part of the vocabulary or **out-of-vocabulary (OOV)** words?
>
> 1. One option is to replace OOV words with a special token, `[UNK].` In this case, all OOV terms will have the same representation for the neural network losing the semantic. 
> 1. A second option is to split OOV words at the character level. This increases the size of the input to the neural language model, adding the challenge of learning the relationship between characters to keep the semantic.
> 1. Sub-word tokenizers, such as BERT WordPiece, provide a solution in between the word token and character split option. It tokenizes OOV words into subwords.
>
> Let's have a look at the `bert-base-uncased` tokenizer:

In [ ]:
# load the bert-base-uncased tokenizer 
tokenizer_uncased = nemo_nlp.modules.get_tokenizer(tokenizer_name="bert-base-uncased")

# get the vocabulary size
print(f"The vocabulary size: {tokenizer_uncased.vocab_size}")

> As an example, take a look at the format tokenization for years with BERT. Years prior to 2021 appear frequently enough in the corpus to be part of the vocabulary, while years in the future are OOV and are split into sub-tokens.
>
> Try it in the cell below using the `tokenizer_uncased.text_to_tokens()` function for various years.

In [ ]:
# Bert tokenizer for years
print(f"Tokenized year: {tokenizer_uncased.text_to_tokens('2019')}")
print(f"Tokenized year: {tokenizer_uncased.text_to_tokens('2020')}")
print(f"Tokenized year: {tokenizer_uncased.text_to_tokens('2021')}")
print(f"Tokenized year: {tokenizer_uncased.text_to_tokens('2022')}")
print(f"Tokenized year: {tokenizer_uncased.text_to_tokens('2023')}")
print(f"Tokenized year: {tokenizer_uncased.text_to_tokens('2030')}")

## Where to next?

Initial attempts at language models were largely statistical, trying to find statistical patterns in texts to draw conclusions. In 2013 and 2014, Ilya Sutskever, one of the co-inventors of AlexNet (and now famous for being one of the OpenAI board members who fired Sam Altman in 2023), started applying deep learning to language models.

### RNNs and LSTMs

In addition to the challenges discussed above, another challenge with language modeling is that the order of words matters. Initial attempts at neural networks for language used Recurrent Neural Networks (RNNs) to try to capture meaning over the sequence of words.

![RNN diagram from Wikimedia Commons](https://upload.wikimedia.org/wikipedia/commons/b/b5/Recurrent_neural_network_unfold.svg)

One challenge that quickly became apparent was that the further apart words were, the harder it was to have their information influence each other. Long Short-Term  Memory (LSTM) networks were developed to try to capture these longer dependencies using remember and forget gates.

![LSTM diagrom from Wikimedia Commons](https://upload.wikimedia.org/wikipedia/commons/9/93/LSTM_Cell.svg)

This helped some, but the context was still hard to propagate beyond relatively short sentences.

### CNNs and Attention

By 2015, people were experimenting with using CNNs (yes, the same CNNs we used in computer vision!). The kernel was one-dimensional scanning across the sequence of words. Attention was also introduced as part of CNN methods.

### Attention is All You Need

As we will see in the next notebook, with the publishing of the 2017 paper introducing transformers, the field was...*transformed*!